In [1]:
!pip install pandas-datareader
import boto3
import s3fs
import sagemaker
from sagemaker import get_execution_role
import time
import numpy as np
import pandas as pd
import json
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import bs4 as bs
import pickle
import requests
import matplotlib.pyplot as plt

    100% |████████████████████████████████| 112kB 2.9MB/s ta 0:00:01   28% |█████████▏                      | 30kB 2.5MB/s eta 0:00:01
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# session, role, bucket
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()             # IAM role to use by SageMaker

s3_bucket = sagemaker_session.default_bucket()
s3_prefix = 'MLEND-Capstone-Project'    

region = sagemaker_session.boto_region_name

s3_data_path = "s3://{}/{}/data".format(s3_bucket, s3_prefix)
s3_output_path = "s3://{}/{}/output".format(s3_bucket, s3_prefix)

image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")


In [3]:
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

    soup = bs.BeautifulSoup(resp.text, "lxml")
    table = soup.find('table', {'class':'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        ticker = ticker[:-1]
        tickers.append(ticker)

    with open('sp500tickers.pickle','wb') as f:
        pickle.dump(tickers, f)
    return tickers

tickers = save_sp500_tickers()
tickers.sort()
tickers.remove('BF.B')
tickers.remove('BRK.B')
tickers.remove('CARR')
tickers.remove('DPZ')
tickers.remove('DXCM')
tickers.remove('OTIS')
tickers.remove('WST')

hm_days = 7

def buy_sell_hold(*args):
    cols = [c for c in args]
    requirement = 0.0235
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0


In [4]:
print('update')

update


In [259]:
from collections import OrderedDict
import pytz

data = OrderedDict()
stock_data_preprocessed = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv("stocks_dfs/{}.csv".format(ticker), index_col=0, parse_dates=['Date'])
    df = df[['Adj Close','Volume']]
    df.fillna(0, inplace=True)
    for i in range(1, hm_days+1):
        df['{}d'.format(i)] = (df['Adj Close'].shift(-i) - df['Adj Close']) / df['Adj Close']
        
    '''df['50MA'] = df['Adj Close'].rolling(50).mean()
    df['25MA'] = df['Adj Close'].rolling(25).mean()
    df['10MA'] = df['Adj Close'].rolling(10).mean()
    df['5MA'] = df['Adj Close'].rolling(5).mean()

    df['50STD'] = df['Adj Close'].rolling(50).std()
    df['25STD'] = df['Adj Close'].rolling(25).std()
    df['10STD'] = df['Adj Close'].rolling(10).std()

    df['50UBB'] = df['50MA'] + (df['50STD'] * 2)
    df['25UBB'] = df['25MA'] + (df['25STD'] * 2)

    df['50LBB'] = df['50MA'] - (df['50STD'] * 2)
    df['25LBB'] = df['25MA'] - (df['25STD'] * 2)'''
    
    data[ticker] = df
    

stock_data_preprocessed = pd.concat(data.values(),keys=tickers,names=['Ticker','Date'])

stock_data_preprocessed = stock_data_preprocessed.swaplevel()
stock_data_preprocessed = stock_data_preprocessed.sort_index()
#stock_data_preprocessed.to_csv('stock_data_preprocessed.csv')

KeyboardInterrupt: 

In [ ]:
stock_data_preprocessed.head()

In [7]:
stock_data_preprocessed['target'] = list(map( buy_sell_hold,
                                               stock_data_preprocessed['1d'.format(ticker)],
                                               stock_data_preprocessed['2d'.format(ticker)],
                                               stock_data_preprocessed['3d'.format(ticker)],
                                               stock_data_preprocessed['4d'.format(ticker)],
                                               stock_data_preprocessed['5d'.format(ticker)],
                                               stock_data_preprocessed['6d'.format(ticker)],
                                               stock_data_preprocessed['7d'.format(ticker)] ))

KeyError: '1d'

In [17]:
stock_data_preprocessed.head()

Adj Close       Volume        1d        2d        3d  \
Date       Ticker                                                         
2010-01-04 A       20.436504    3815500.0 -0.010862 -0.014377 -0.015655   
           AAL      4.496876    9837300.0  0.113208  0.067086  0.098533   
           AAP     39.293575    1701700.0 -0.005943  0.002724  0.002476   
           AAPL    26.538483  123432400.0  0.001729 -0.014205 -0.016027   
           ABC     22.813559    2455900.0 -0.007134 -0.016522 -0.032294   

                         4d        5d        6d        7d  target  
Date       Ticker                                                  
2010-01-04 A      -0.015975 -0.015336 -0.027157 -0.019489      -1  
           AAL     0.077568  0.056604  0.064990  0.148847       1  
           AAP     0.006439 -0.003467 -0.020802 -0.007182       0  
           AAPL   -0.009485 -0.018223 -0.029391 -0.015700      -1  
           ABC    -0.021780 -0.010890 -0.004131  0.015396      -1

In [6]:
stock_data_preprocessed.drop(columns=['1d', '2d','3d','4d','5d','6d','7d'], inplace=True)
stock_data_preprocessed.replace([np.inf, -np.inf], np.nan, inplace = True)
stock_data_preprocessed.fillna(0, inplace=True)

KeyError: "['1d' '2d' '3d' '4d' '5d' '6d' '7d'] not found in axis"

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
stock_data_preprocessed[['Adj Close', 'Volume']] = scaler.fit_transform(stock_data_preprocessed[['Adj Close', 'Volume']])
stock_data_preprocessed.to_csv('stock_data_preprocessed.csv')

KeyError: u"None of [Index([u'Adj Close', u'Volume'], dtype='object')] are in the [columns]"

In [260]:
stock_data_preprocessed = pd.read_csv('stock_data_preprocessed.csv',parse_dates=True, index_col=[0,1])
print("-1: ", stock_data_preprocessed[stock_data_preprocessed['target']==-1].size)
print(" 0: ", stock_data_preprocessed[stock_data_preprocessed['target']==0].size)
print(" 1: ", stock_data_preprocessed[stock_data_preprocessed['target']==1].size)
stock_data_preprocessed.head(10)


('-1: ', 1109910)
(' 0: ', 1087758)
(' 1: ', 1400007)


Adj Close    Volume  target
Date       Ticker                             
2010-01-04 A        0.005248  0.004438      -1
           AAL      0.001154  0.011443       1
           AAP      0.010092  0.001980       0
           AAPL     0.006816  0.143586      -1
           ABC      0.005859  0.002857      -1
           ABMD     0.002244  0.000173      -1
           ABT      0.004662  0.012597       1
           ACN      0.008660  0.004246       0
           ADBE     0.009526  0.005479      -1
           ADI      0.006230  0.002446      -1

In [20]:
# we use 2 hour frequency for the time series
freq = 'D'

# we predict for 7 days
prediction_length = 1

# we also use 7 days as context length, this is the number of state updates accomplished before making predictions
context_length = 50

end_training = pd.Timestamp("2018-12-31 00:00:00", freq=freq)

timeseries = []
for ID,ticker in list(enumerate(tickers)):
    ticker = stock_data_preprocessed.loc[(slice(None), ticker), :]
    if ticker.index[0][0]<end_training:
        timeseries.append(ticker)


In [108]:
training_data = [
    {
            "start": str(ts.index[0][0]),
            "target": ts['target'][ts.index[0][0]:end_training].tolist(), # We use -1, because pandas indexing includes the upper bound 
            "dynamic_feat": ts[['Adj Close','Volume']][ts.index[0][0]:end_training].values.T.tolist()
    }
    for ts in timeseries
]
print(len(training_data))

491


In [109]:
num_test_windows = 10

test_data_2 = [
    {
        "start": str(ts.index[0][0]),
        "target": ts['target'][ts.index[0][0]:end_training + (2*k * prediction_length)].tolist(),
        "dynamic_feat": ts[['Adj Close','Volume']][ts.index[0][0]:end_training + (2*k * prediction_length)].values.T.tolist()
    }
    for k in range(1, num_test_windows + 1) 
    for ts in timeseries
]
print(len(test_data))


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`


4910


In [112]:
def write_json_dataset(filename, data): 
    with open(filename, 'wb') as f:
        # for each of our times series, there is one JSON line
        for d in data:
            json_line = json.dumps(d) + '\n'
            json_line = json_line.encode('utf-8')
            f.write(json_line)            

In [113]:
%%time
write_json_dataset("train.json", training_data)
write_json_dataset("test.json", test_data)

CPU times: user 2min 21s, sys: 1.35 s, total: 2min 23s
Wall time: 2min 33s


In [114]:
s3 = boto3.resource('s3')
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{}/{} already exists.\nSet override to upload anyway.\n'.format(s3_bucket, s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)

In [115]:
%%time
copy_to_s3("train.json", s3_data_path + "/train/train.json")
copy_to_s3("test.json", s3_data_path + "/test/test.json")

Uploading file to s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data/train/train.json
Uploading file to s3://sagemaker-us-east-2-017500148529/MLEND-Capstone-Project/data/test/test.json
CPU times: user 4 s, sys: 1.36 s, total: 5.35 s
Wall time: 23.9 s


In [116]:
s3filesystem = s3fs.S3FileSystem()
with s3filesystem.open(s3_data_path + "/train/train.json", 'rb') as fp:
    print(fp.readline().decode("utf-8")[:100] + "...")

{"start": "2010-01-04 00:00:00", "target": [-1, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -...


In [117]:
estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker_session,
    image_name=image_name,
    role=role,
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='MLEND-Capstone-Project',
    output_path=s3_output_path
)

In [118]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "10",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "num_dynamic_feat": 'auto',
}

In [119]:
estimator.set_hyperparameters(**hyperparameters)

In [121]:
%%time
data_channels = {
    "train": s3_data_path + "/train/train.json",
    "test": s3_data_path + "/test/test.json"
}

estimator.fit(inputs=data_channels, wait=True)

2020-06-01 00:27:34 Starting - Starting the training job...
2020-06-01 00:27:37 Starting - Launching requested ML instances......
2020-06-01 00:28:40 Starting - Preparing the instances for training......
2020-06-01 00:30:00 Downloading - Downloading input data
2020-06-01 00:30:00 Training - Downloading the training image..Arguments: train
[06/01/2020 00:30:16 INFO 140489765541696] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': u''}
[06/01/2020 00:30:16 I

In [ ]:
#predictor = estimator.deploy(
#    initial_instance_count=1,
#    instance_type='ml.m4.xlarge',
#    content_type="application/json")

In [261]:
def encode_request(instance, num_samples, quantiles):
        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')


In [262]:
predictor = sagemaker.predictor.RealTimePredictor(endpoint='MLEND-Capstone-Project-2020-05-31-23-12-35-007')

In [391]:
def get_stock_prediction(ticker,date):
    try:
        date_pred = pd.Timestamp(date, freq='D')
        date_start = date_pred-50
        pred_df = stock_data_preprocessed.loc[(slice(str(date_start),str(date_pred)), ticker), :]
        result_df = pred_df.loc[(slice(str(date_pred),str(date_pred)), ticker), :]

        pred = {
                "start": str(date_pred),
                "target": pred_df['target'][date_start:date_pred-1].tolist(),
                "dynamic_feat": pred_df[['Adj Close','Volume']][date_start:date_pred].values.T.tolist()
        }

        req = encode_request(instance=pred, num_samples=50, quantiles=['0.1', '0.5', '0.9'])
        res = predictor.predict(req)

        prediction_data = json.loads(res.decode('utf-8'))
        pred = round(prediction_data['predictions'][0]['quantiles']['0.5'][0])
        result_df['prediction'] = pred
        return result_df
    except:
        print('{} did not trade today.'.format(ticker))

In [ ]:
get_stock_prediction('AAPL', '2019-01-23')

In [415]:
date = pd.Timestamp("2019-01-19", freq='D')
i = 0
target = []
prediction = []

#for i in range(200):
d = date+i
print(get_stock_prediction('AAPL', str(d))['target'].values[0])
print(int(get_stock_prediction('AAPL', str(d))['prediction'].values[0]))

   # prediction.append(get_stock_prediction('AAPL', str(d))['prediction'])


AAPL did not trade today.


/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
/home/ec2-user/anaconda3/envs/amazonei_mxnet_p27/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`


TypeError: 'NoneType' object has no attribute '__getitem__'

In [405]:
target

[Date        Ticker
 2019-01-16  AAPL      0
 Name: target, dtype: int64, Date        Ticker
 2019-01-17  AAPL      0
 Name: target, dtype: int64, Date        Ticker
 2019-01-18  AAPL     -1
 Name: target, dtype: int64]